In [1]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page in reader.pages:
            text += page.extract_text()
    return text

pdf_path = r'C:\Users\pc\Desktop\ChatLaw\data\code_penal.pdf'
text = extract_text_from_pdf(pdf_path)

In [2]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Charger le modèle d'encodage
model = SentenceTransformer('all-MiniLM-L6-v2')

# Diviser le texte en phrases ou paragraphes
sentences = text.split('\n')

# Encoder les phrases
sentence_embeddings = model.encode(sentences)

# Créer un index FAISS
dimension = sentence_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(sentence_embeddings)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
c:\Users\pc\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
import ollama

def search_and_answer(question, index, sentences, model, top_k=3):
    # Encoder la question
    question_embedding = model.encode([question])
    
    # Rechercher les phrases les plus proches
    distances, indices = index.search(question_embedding, top_k)
    
    # Récupérer les phrases pertinentes
    relevant_sentences = [sentences[i] for i in indices[0]]
    
    # Préparer le contexte pour le modèle de langage
    context = "\n".join(relevant_sentences)
    
    # Générer une réponse avec Mistral via Ollama
    response = ollama.generate(model='mistral', prompt=f"Contexte:\n{context}\n\nQuestion: {question}\nRéponse:")
    
    return response['response']

# Exemple d'utilisation
question = "Quelle est la peine pour le vol selon le Code Pénal marocain?"
answer = search_and_answer(question, index, sentences, model)
print(answer)

 Selon le Code Pénal marocain, la peine pour le vol dépend de l'importance du bien volé. En général, un vol avec simple dépassement de limite peut être puni d'une peine de prison allant jusqu'à 3 ans et d'une amende. Si le vol implique une arme ou un acte de violence, la peine peut aller jusqu'à 10 ans de prison et jusqu'à 5 ans de prison si le bien volé est apparemment inoffensif.
